In [61]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM

# Adding Special Tokens to tokenizer

In [62]:
tokenizer = LlamaTokenizer.from_pretrained("maritaca-ai/sabia-7b")

In [63]:
inst = "</INST>"
inst_id = tokenizer(inst, return_tensors="pt")
inst_id

{'input_ids': tensor([[    1,  1533, 25580, 29958]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [64]:
special_tokens_dict = {"additional_special_tokens": ["<INST>", "</INST>"]}

tokenizer.add_special_tokens(special_tokens_dict)

2

In [65]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<INST>', '</INST>']

In [66]:
test = "<INST>Eu quero que você me gere uma música brasileira</INST>"
input_ids = tokenizer(test, return_tensors="pt")
input_ids

{'input_ids': tensor([[    1, 32000, 16430,   439,  1489,   712,  7931, 30037,   592,   330,
           406,  3672, 15433, 22905,  3055, 32001]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [67]:
tokenizer.decode(list(input_ids['input_ids'][0]), skip_special_tokens=True)

'Eu quero que você me gere uma música brasileira'

In [98]:
tokenizer.add_special_tokens({"pad_token": "<PAD>"})

1

In [101]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<PAD>', '<INST>', '</INST>']

# Loading Dataset

In [68]:
import datasets

dataset = datasets.load_dataset("dominguesm/Canarim-Instruct-PTBR-Dataset")
dataset

Generating test split: 100%|██████████| 1519/1519 [00:00<00:00, 284718.58 examples/s]


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 316413
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1519
    })
})

In [79]:
dataset['train'][1234]

{'instruction': 'Nomeie as capitais dos três países seguintes',
 'input': 'ndia, Canadá, Egito',
 'output': 'ndia: Nova Deli. Canadá: Ottawa. Egito: Cairo.'}

In [89]:
def format_prompt_input(row):
    row['prompt'] = \
        f"""Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação.\n\n### Instrução:\n{row["instruction"]}\n\n### Entrada:\n{row['input']}\n\n### Resposta:\n"""

    return row

def format_prompt_no_input(row):
    row['prompt'] = \
        f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.\n\n### Instrução:\n{row["instruction"]}\n\n### Resposta:\n"""

    return row

def create_prompt(row):
    return format_prompt_no_input(row) if row['input'] == "" else format_prompt_input(row)

In [90]:
prompted_dataset = dataset.map(create_prompt)

Map: 100%|██████████| 1519/1519 [00:00<00:00, 22336.88 examples/s]


In [91]:
prompted_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 316413
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 1519
    })
})

In [92]:
print(prompted_dataset['train'][1234]['prompt'])

Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação.

### Instrução:
Nomeie as capitais dos três países seguintes

### Entrada:
ndia, Canadá, Egito

### Resposta:



In [93]:
print(prompted_dataset['train'][0]['prompt'])

Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.

### Instrução:
Dê três dicas para se manter saudável.

### Resposta:

